Все работает, но не все в итоге полноценно запущено, в ДЗ 4 точно так же все и оно работает.

In [1]:
import os
import string

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument, TaggedLineDocument

from nltk.tokenize import word_tokenize
punkt = string.punctuation+'»«–…'

from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from judicial_splitter import splitter
from tqdm import tqdm_notebook as tqdm

import json

import numpy as np
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from itertools import zip_longest

In [4]:
from heapdict import heapdict
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

# Basic

In [5]:
def preprocessing(text, stop=False):
    global word_tokenize, stop_words, punkt
    text = re.sub(r"([a-zа-я0-9])(.)([A-ZА-Я0-9])", r"\1\2 \3", text)
    text = word_tokenize(text)
    new_text= []
    for word in text:
        word = word.strip(punkt)
        if word:
            if word in punkt: continue
            elif word.isdigit(): continue
            elif stop and word in stop_words: continue
            else: new_text.append(morph.parse(word)[0].normal_form)
    return new_text

In [6]:
def write(path):
    global qa_corpus
    with open('d2v_answers_3.txt','w', encoding = 'utf-8') as answers:
        with open ('d2v_indexes.txt', 'w', encoding = 'utf-8') as indexes:
            for root, dirs, files in list(os.walk(path)):
                for file in tqdm(files):
                    with open (os.path.join(root, file), 'r', encoding='utf-8') as f:
                        article = f.read()
                    for chunk in splitter(article, 3):
                        answer = ' '.join(preprocessing(chunk))
                        if len(answer) > 5:
                            answers.write(answer+'\n')
                            indexes.write(str(file.split('.')[-2])+'\n')

Записать всё в предобработанном варианте, чтобы быстрее проверять.

In [7]:
write('/home/dkbrz/data/article/')

# D2V

Итерирование по файлу вместо хранения при обучении.

In [8]:
def docs():
    for key, value in enumerate(zip_longest(open('d2v_indexes.txt','r'), open('d2v_answers_3.txt','r'))):
        yield TaggedDocument(words=value[1].strip().split(), tags=[int(value[0].strip())])

In [15]:
docs()

<generator object docs at 0x7fd5800a6200>

In [7]:
class LabeledLineSentence(object):
    def __init__(self, name):
        self.name = name
    def __iter__(self):
        for key, value in enumerate(zip_longest(open('d2v_indexes.txt','r'), open('d2v_answers_3.txt','r'))):
            yield TaggedDocument(words=value[1].strip().split(), tags=[int(value[0].strip())])

In [8]:
d2v_model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, seed = 23,
                min_alpha=0.025, epochs=200, workers=8, dm=1)

%time d2v_model.build_vocab(LabeledLineSentence(''))
print (len(d2v_model.wv.vocab))
%time d2v_model.train(LabeledLineSentence(''), total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

CPU times: user 38.4 s, sys: 526 ms, total: 39 s
Wall time: 39 s
75401
CPU times: user 14h 32min 30s, sys: 1h 6min 36s, total: 15h 39min 6s
Wall time: 6h 36min 13s


In [9]:
d2v_model.save('d2v')

In [10]:
d2v_model = Doc2Vec.load('d2v')

In [12]:
def d2v_search(query, n=5):
    global d2v_model
    d2v_model.random.seed(23)
    d2v_vector = d2v_model.infer_vector(query.strip().split(), epochs=1000)
    result = {i[0]: i[1] for i in d2v_model.docvecs.most_similar(positive = [d2v_vector], topn=n)}
    return dict(result)

In [ ]:
import json
def save_d2v_base():
    global d2v_model
    with open('d2v_vectors','w', encoding = 'utf-8') as vectors:
        for line in tqdm(open('d2v_answers_3.txt','r')):
            d2v_model.random.seed(23)
            d2v = d2v_model.infer_vector(line.strip().split())
            d2v = d2v.tolist()
            vectors.write(json.dumps(d2v)+'\n')
%time save_d2v_base()

In [21]:
def d2v_search(query, n=10):
    global d2v_model
    result = heapdict()
    d2v_model.random.seed(23)
    q_d2v = d2v_model.infer_vector(query.strip().split())
    for key, value in enumerate(zip_longest(open('d2v_indexes.txt','r'), open('d2v_vectors','r'))):
        d2v = json.loads(value[1])
        x = cosine_similarity([q_d2v], [d2v])[0][0]
        #print (x)
        ordinal = int(value[0])
        if ordinal in result:
            if result[ordinal] < x:
                result[ordinal] = x
        else:
            if len(result) == n:
                z = result.peekitem()
                if x > z[1]:
                    result.popitem()
                    result[ordinal] = x
            else:
                result[ordinal] = x
    return dict(result)

# W2V

In [ ]:
def get_w2v_vectors(text, k=300, prep=True):
    """Получает вектор документа"""
    
    global w2v_model, stop_words, word_tokenize
    
    if prep:
        arr_text = preprocessing(text, stop=True)
    else:
        arr_text = text.split()
    n = 0
    vector = np.array([0]*300)
    
    for word in arr_text:
        if word not in stop_words:
            try:
                vec = np.array(w2v_model.wv[word])
                n += 1 
                vector = vector + vec
            except:
                continue
            
    if n > 0: vector = vector / n
    
    return vector

In [ ]:
w2v_model = Word2Vec.load('/home/dkbrz/data/rusvectores/araneum_none_fasttextcbow_300_5_2018.model')

In [28]:
def w2v_search(query, n=10):
    result = heapdict()
    q_w2v = get_w2v_vectors(query, k=300, prep=False)
    for key, value in enumerate(zip_longest(open('d2v_indexes.txt','r'), open('w2v_vectors','r'))):
        w2v = json.loads(value[1])
        x = cosine_similarity([q_w2v], [w2v])[0][0]
        #print (x)
        ordinal = int(value[0])
        if ordinal in result:
            if result[ordinal] < x:
                result[ordinal] = x
        else:
            if len(result) == n:
                z = result.peekitem()
                if x > z[1]:
                    result.popitem()
                    result[ordinal] = x
            else:
                result[ordinal] = x
    return dict(result)

In [ ]:
import json
def save_w2v_base():
    with open('w2v_vectors','w', encoding = 'utf-8') as vectors:
        for line in tqdm(open('d2v_answers_3.txt','r')):
            w2v = get_w2v_vectors(line.strip(), k=300, prep=False)
            w2v = w2v.tolist()
            vectors.write(json.dumps(w2v)+'\n')
%time save_w2v_base()

# OKAPI

In [47]:
def write():
    global qa_corpus
    with open('ok_answers.txt','w', encoding = 'utf-8') as answers:
                for key, value in enumerate(tqdm(qa_corpus)):
                        answer = ' '.join(preprocessing(value[1]))
                        answers.write(answer+'\n')
write()

In [36]:
from collections import Counter
import math
from collections import defaultdict
from itertools import islice

In [58]:
def get_term_doc_matrix():
    n = 1384
    dictionary = {}
    term_doc_matrix = []
    for key, item in enumerate(open('ok_answers.txt','r')):
        text = Counter(item.strip().split())
        for word in text:
            if word in dictionary:
                term_doc_matrix[dictionary[word]][key] += text[word]
            else:
                dictionary[word] = len(dictionary)
                term_doc_matrix.append(np.zeros(n))
                term_doc_matrix[dictionary[word]][key] += text[word]
    return dictionary, term_doc_matrix

def inverted_index(dictionary, term_doc_matrix) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    result = {}
    for word in dictionary:
        result[word] = {key: int(value) for key, value in enumerate(term_doc_matrix[dictionary[word]]) if value > 0}
    return result

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log(1 + (N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

def compute_sim(word, index, dictionary, term_doc_matrix, doc_length, avgdl, N) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    if word in dictionary:
        n = len(index[word])
        result = {}
        for doc in index[word]:
            qf = term_doc_matrix[dictionary[word]][doc]/doc_length[doc]
            #qf = term_doc_matrix[dictionary[word]][doc]
            score = score_BM25(qf, doc_length[doc], avgdl, k1, b, N, n)
            result[doc] = score
        return result
    else:
        return {}

def get_okapi(query, n = 30) -> float:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    global index, dictionary, term_doc_matrix, doc_length, avgdl, N
    query = query.strip().split()
    result = defaultdict(int)
    for word in query:
        current = compute_sim(word, index, dictionary, term_doc_matrix, doc_length, avgdl, N)
        for doc in current:
            result[doc] += current[doc]
    return {i[0]:i[1] for i in sorted(result.items(), key = lambda x: x[1], reverse = True)[:n]}

In [49]:
dictionary, term_doc_matrix = get_term_doc_matrix()
index = inverted_index(dictionary, term_doc_matrix)

k1 = 2.0
b = 0.75

doc_length = {}
for key, value in enumerate(np.transpose(term_doc_matrix)):
    doc_length[key] = sum(value)

avgdl = sum(doc_length.values())/len(doc_length)
N = len(doc_length)

In [115]:
def general_search(query, n=10, prep=True):
    if prep: query = ' '.join(preprocessing(query))
    w2v = w2v_search(query, n=250)
    d2v = d2v_search(query, n=250)
    okapi = get_okapi(query, n=250)
    candidates = set(w2v) | set(d2v) | set(okapi)
    result = heapdict()
    #print (candidates)
    for i in candidates:
        if i in okapi: x = okapi[i]
        else: x = 1
        coef = 0
        if i in w2v: coef += w2v[i]
        if i in d2v: coef += d2v[i]
        coef = coef*(1+math.log(1 +x))
        if len(result) == n and coef > result.peekitem()[1]:
            result.popitem()
            result[i] = coef*(1+math.log(x))
        elif len(result) < n: result[i] = coef*(1+math.log(x))
    return dict(result)

In [118]:
def general_search(query, n=10, prep=True):
    if prep: query = ' '.join(preprocessing(query))
    w2v = w2v_search(query, n=250)
    #d2v = d2v_search(query, n=250)
    okapi = get_okapi(query, n=250)
    #candidates = set(w2v) | set(d2v) | set(okapi)
    candidates = set(w2v) | set(okapi)
    result = heapdict()
    #print (candidates)
    for i in candidates:
        if i in okapi: x = okapi[i]
        else: x = 1
        coef = 0
        if i in w2v: coef += w2v[i]
        #if i in d2v: coef += d2v[i]
        coef = coef*(1+math.log(1 +x))
        if len(result) == n and coef > result.peekitem()[1]:
            result.popitem()
            result[i] = coef*(1+math.log(x))
        elif len(result) < n: result[i] = coef*(1+math.log(x))
    return dict(result)

In [ ]:
def general_search(query, n=10, prep=True):
    if prep: query = ' '.join(preprocessing(query))
    w2v = w2v_search(query, n=500)
    #d2v = d2v_search(query, n=250)
    okapi = get_okapi(query, n=500)
    #candidates = set(w2v) | set(d2v) | set(okapi)
    candidates = set(w2v) | set(okapi)
    result = heapdict()
    #print (candidates)
    for i in candidates:
        if i in okapi: x = okapi[i]
        else: x = 1
        coef = 0
        if i in w2v: coef += w2v[i]
        #if i in d2v: coef += d2v[i]
        coef = coef*(1+math.log(1 +x))
        if len(result) == n and coef > result.peekitem()[1]:
            result.popitem()
            result[i] = coef*(1+math.log(x))
        elif len(result) < n: result[i] = coef*(1+math.log(x))
    return dict(result)

In [ ]:
def general_search(query, n=10, prep=True):
    if prep: query = ' '.join(preprocessing(query))
    w2v = w2v_search(query, n=500)
    #d2v = d2v_search(query, n=250)
    okapi = get_okapi(query, n=500)
    #candidates = set(w2v) | set(d2v) | set(okapi)
    candidates = set(w2v) | set(okapi)
    result = heapdict()
    #print (candidates)
    for i in candidates:
        if i in okapi: x = okapi[i]
        else: x = 1
        coef = 0
        if i in w2v: coef += w2v[i]
        #if i in d2v: coef += d2v[i]
        coef = math.exp(coef)*x
        if len(result) == n and coef > result.peekitem()[1]:
            result.popitem()
            result[i] = coef*(1+math.log(x))
        elif len(result) < n: result[i] = coef*(1+math.log(x))
    return dict(result)

In [1]:
def best(result):
    i = max(result, key=result.get)
    with open('/home/dkbrz/data/article/{}.txt'.format(i), 'r', encoding='utf-8') as f:
        return f.read()

In [4]:
best({'4085':2, '345':50})

'\n\nПЕРЕСМОТРЕ Судья Высшего Арбитражного Суда Российской Федерации Бондаренко С.П., рассмотрев заявление общества с ограниченной ответственностью "Нева-Ресурс" от 10.11.2011 б/н о пересмотре в порядке надзора решения Арбитражного суда города Санкт-Петербурга и Ленинградской области от 02.12.2010 по делу N А56-31154/2010 и постановления Федерального арбитражного суда Северо-Западного округа от 21.09.2011 по тому же делу, установил: заявление подано с соблюдением требований, предусмотренных статьями 292, 294 Арбитражного процессуального кодекса Российской Федерации. Руководствуясь статьей 295 Арбитражного процессуального кодекса Российской Федерации, судья Высшего Арбитражного Суда Российской Федерации определил: принять заявление общества с ограниченной ответственностью "Нева-Ресурс" от 10.11.2011 б/н о пересмотре в порядке  надзора решения Арбитражного суда города Санкт-Петербурга  и Ленинградской области от 02.12.2010 по делу N А56-31154/2010 и постановления Федерального арбитражног